In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
import torch
print(torch.cuda.device_count())
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from random import randint

import tqdm

from rotation_utils import random_orthogonal_matrix
from hadamard_utils import random_hadamard_matrix, apply_exact_had_to_linear
from quant_utils import ActQuantWrapper, find_qlayers, add_actquant

import utils
import model_utils
import data_utils
import transformers
import quant_utils
import rotation_utils
import gptq_utils
import eval_utils
import hadamard_utils

import rotated_llama

8


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
R1 = torch.load('R1.pt')

In [4]:
learned_R1 = torch.load('/ceph/echoi/codes/QuaRot/fake_quant/rotations/SpinQuant_m_gptq_b2_R1.pt')

In [5]:
((R1 - learned_R1) < 1e-4).sum() / R1.numel()

tensor(0.5452)

In [6]:
print((torch.matmul(R1, R1.T) - torch.eye(R1.size(0))).sum().item())

0.0


In [7]:
R2s = torch.load('R2s.pt')
learned_R2s = []

for i in range(32):
    R2 = torch.load(f'rotations/SpinQuant_m_gptq_b2_R2_{i}.pt')
    print(((R2s[i] - R2) < 1e-4).sum() / R2.numel())

tensor(0.5060)
tensor(0.5098)
tensor(0.5211)
tensor(0.5203)
tensor(0.5075)
tensor(0.5137)
tensor(0.5125)
tensor(0.5068)
tensor(0.4995)
tensor(0.5068)
tensor(0.5054)
tensor(0.4968)
tensor(0.5099)
tensor(0.5151)
tensor(0.5022)
tensor(0.5079)
tensor(0.5039)
tensor(0.5012)
tensor(0.5147)
tensor(0.5045)
tensor(0.5046)
tensor(0.5077)
tensor(0.5051)
tensor(0.5196)
tensor(0.5108)
tensor(0.5031)
tensor(0.5071)
tensor(0.5009)
tensor(0.5020)
tensor(0.5027)
tensor(0.5002)
tensor(0.5068)


In [18]:
R2s.size()

torch.Size([32, 128, 128])

In [20]:
for i in range(len(R2s)):
    print((torch.matmul(R2s[i], R2s[i].T) - torch.eye(R2s[i].size(0))).sum().item())

4.381253829032868e-06
4.381253832105949e-06
4.381253831644226e-06
4.381253830711715e-06
4.381253830295746e-06
4.38125383091246e-06
4.38125383249077e-06
4.381253829978891e-06
4.381253831519326e-06
4.381253830038432e-06
4.381253829899381e-06
4.381253828759852e-06
4.381253830187057e-06
4.381253832988036e-06
4.381253829727632e-06
4.381253830506247e-06
4.381253829962314e-06
4.381253829684758e-06
4.3812538310376335e-06
4.381253829330392e-06
4.381253829407568e-06
4.381253830940215e-06
4.381253830244776e-06
4.381253829419293e-06
4.381253829949258e-06
4.381253828830792e-06
4.381253831584867e-06
4.381253831533204e-06
4.3812538313707e-06
4.381253830409103e-06
4.381253830225992e-06
4.381253831102901e-06


In [12]:
for R in [R1] + R2s:
    print((torch.matmul(R, R.T) - torch.eye(R.size(0))).sum().item())

-8.191814676716455e-05
-2.555238878707078e-06
-2.5665923312333804e-06
-2.560000028607973e-06
-2.559999995922207e-06
-2.5599999623935295e-06
-2.5599999707469168e-06
-2.559999946718113e-06
-2.5600002815259817e-06
-2.560000858826697e-06
-2.5600000005435076e-06
-2.5599999631534595e-06
-2.560007021463766e-06
-2.560000381007865e-06
-2.5600000345841046e-06
-2.559987751149775e-06
-2.5600008967903782e-06
-2.560000014940212e-06
-2.5600000586382385e-06
-2.559999950954702e-06
-2.5599999533982548e-06
-2.5599999366024706e-06
-2.5599997960222475e-06
-2.5600053767368696e-06
-2.5599999238307414e-06
-2.5599999655497386e-06
-2.5599999905528184e-06
-2.559998568072487e-06
-2.5599996680228162e-06
-2.559999948066079e-06
-2.5600015085024028e-06
-2.5600007700122736e-06
-2.5600000223906066e-06


In [2]:
args = utils.parser_gen('--model meta-llama/Llama-2-7b-hf --rotate --a_bits 4 --v_bits 4 --k_bits 4 --w_bits 16 --w_clip --bsz 1'.split())

Arguments: 
{'a_asym': False,
 'a_bits': 16,
 'a_clip_ratio': 1.0,
 'a_groupsize': -1,
 'act_order': False,
 'bsz': 1,
 'cal_dataset': 'wikitext2',
 'capture_layer_io': False,
 'distribute': False,
 'eval_dataset': 'wikitext2',
 'fp32_had': False,
 'hf_token': None,
 'int8_down_proj': False,
 'k_asym': False,
 'k_bits': 4,
 'k_clip_ratio': 1.0,
 'k_groupsize': -1,
 'k_pre_rope': False,
 'layer_idx': 10,
 'learn_r1': True,
 'learn_r2': True,
 'lm_eval': False,
 'lm_eval_batch_size': 128,
 'load_qmodel_path': None,
 'model': 'meta-llama/Llama-2-7b-hf',
 'momentum': 0.0,
 'nsamples': 128,
 'percdamp': 0.01,
 'prefix_r': '',
 'rotate': True,
 'rotate_mode': 'hadamard',
 'rotation_seed': -1,
 'save_name': '20240621_092322',
 'save_path': '/ceph/echoi/codes/QuaRot/fake_quant/experiments/meta-llama/Llama-2-7b-hf/20240621_092322',
 'save_qmodel_path': None,
 'seed': 0,
 'tasks': ['piqa',
           'hellaswag',
           'arc_easy',
           'arc_challenge',
           'winogrande',
       

In [3]:
trainloader = data_utils.get_loaders(
    args.cal_dataset, nsamples=args.nsamples,
    seed=args.seed, model=args.model,
    seqlen=2048, eval_mode=True
)

from transformers.trainer_pt_utils import LabelSmoother
label_smoother = LabelSmoother(0.0)

In [4]:
transformers.set_seed(args.seed)
model = model_utils.get_model(args.model, args.hf_token)
model.eval()
for p in model.parameters():
    p.requires_grad = False
    
utils.distribute_model(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.74it/s]
---> Loading meta-llama/Llama-2-7b-hf Model with seq_len: 2048


{0: 50758680576, 1: 50758680576, 2: 50758680576, 3: 50758680576, 4: 50758680576, 5: 50758680576, 6: 50758680576, 7: 50758680576, 'cpu': 238095507456}


GPU memory (from distribute_model): 12.63 -> 12.63 GB (0.00 GB)


In [5]:
from rotated_llama import RotatedLlamaForCausalLM
with torch.no_grad():
    rotated_model = RotatedLlamaForCausalLM(model.config, model)
#del model
rotated_model.eval()
rotation_utils.fuse_layer_norms(rotated_model)
for p in rotated_model.parameters():
    p.requires_grad_(False)

In [6]:
quant_utils.add_actquant(
    rotated_model,
    layers=[nn.Linear,
            ActQuantWrapper,
            rotated_llama.RotatedLinear,
            rotated_llama.RotatedOVProj]
)

qlayers = quant_utils.find_qlayers(
    rotated_model.model,
    layers=[nn.Linear,
            ActQuantWrapper,
            rotated_llama.RotatedLinear,
            rotated_llama.RotatedOVProj]
)

for name in qlayers:
    if 'down_proj' in name:
        had_K, K = hadamard_utils.get_hadK(rotated_model.config.intermediate_size)
        hadamard_utils.apply_exact_had_to_linear(qlayers[name].module, had_dim=-1, output=False)
        qlayers[name].online_full_had = True
        qlayers[name].had_K = had_K
        qlayers[name].K = K
        qlayers[name].fp32_had = args.fp32_had


if args.w_bits < 16:
    save_dict = {}
    if args.load_qmodel_path: # Load Quantized Rotated Model
        assert args.rotate, "Model should be rotated to load a quantized model!"
        assert not args.save_qmodel_path, "Cannot save a quantized model if it is already loaded!"
        print("Load quantized model from ", args.load_qmodel_path)
        save_dict = torch.load(args.load_qmodel_path)
        rotated_model.load_state_dict(save_dict["model"])
        
    elif not args.w_rtn: # GPTQ Weight Quantization
        assert "llama" in args.model, "Only llama is supported for GPTQ!"
        
        trainloader = data_utils.get_loaders(
            args.cal_dataset, nsamples=args.nsamples,
            seed=args.seed, model=args.model,
            seqlen=model.seqlen, eval_mode=False
        )
        quantizers = gptq_utils.gptq_fwrd(
            rotated_model,
            trainloader,
            utils.DEV,
            args,
            [rotated_llama.RotatedLinear, rotated_llama.RotatedOVProj])
        save_dict["w_quantizers"] = quantizers
    else: # RTN Weight Quantization
        quantizers = gptq_utils.rtn_fwrd(rotated_model, utils.DEV, args)
        save_dict["w_quantizers"] = quantizers
        
    if args.save_qmodel_path:
        save_dict["model"] = model.state_dict()
        torch.save(save_dict, args.save_qmodel_path)


if args.a_bits < 16 or args.v_bits < 16:
    qlayers = quant_utils.find_qlayers(rotated_model, layers=[quant_utils.ActQuantWrapper])
    down_proj_groupsize = -1
    if args.a_groupsize > 0 and "llama" in args.model:
        down_proj_groupsize = utils.llama_down_proj_groupsize(rotated_model, args.a_groupsize)
    
    for name in qlayers:            
        layer_input_bits = args.a_bits
        layer_groupsize = args.a_groupsize
        layer_a_sym = not(args.a_asym)
        layer_a_clip = args.a_clip_ratio
        
        if 'v_proj' in name and args.v_bits < 16: #Set the v_proj precision
            qlayers[name].out_quantizer.configure(bits=args.v_bits,
                                            groupsize=args.v_groupsize,
                                            sym=not(args.v_asym),
                                            clip_ratio=args.v_clip_ratio)
        
        if 'lm_head' in name: #Skip lm_head quantization   
            layer_input_bits = 16
        
        if 'down_proj' in name: #Set the down_proj precision
            if args.int8_down_proj:
                layer_input_bits = 8
            layer_groupsize = down_proj_groupsize

            
        qlayers[name].quantizer.configure(bits=layer_input_bits,
                                            groupsize=layer_groupsize,
                                            sym=layer_a_sym,
                                            clip_ratio=layer_a_clip)


if args.k_bits < 16:
    if args.k_pre_rope:
        raise NotImplementedError("Pre-RoPE quantization is not supported yet!")
    else:
        rope_function_name = model_utils.get_rope_function_name(rotated_model)
        layers = model_utils.get_layers(rotated_model)
        k_quant_config = {'k_bits':args.k_bits, "k_groupsize": args.k_groupsize,
                                        "k_sym": not(args.k_asym), "k_clip_ratio": args.k_clip_ratio}
        for layer in layers:
            rotation_utils.add_qk_rotation_wrapper_after_function_call_in_forward(
                        layer.self_attn, 
                        rope_function_name, 
                        config=rotated_model.config,
                        **k_quant_config)

In [7]:
utils.distribute_model(rotated_model)

{0: 45612269568, 1: 45809401856, 2: 45809401856, 3: 45809401856, 4: 45809401856, 5: 45809401856, 6: 43827593216, 7: 50758680576, 'cpu': 237826465792}


GPU memory (from distribute_model): 26.45 -> 26.45 GB (0.00 GB)


In [8]:
o = model(trainloader['input_ids'][:, :2048])

In [9]:
q1 = random_hadamard_matrix(model.config.hidden_size, utils.DEV).to(dtype=torch.float64)
q2s = []
for i in range(32):
    q2s.append(random_hadamard_matrix(model.config.hidden_size // model.config.num_attention_heads, utils.DEV).to(dtype=torch.float64))
q2s = torch.stack(q2s)


In [10]:
r = rotated_model(trainloader['input_ids'][:, :2048], R1=q1, R2s=q2s)

In [11]:
o['logits']

tensor([[[-1.2828e+01, -7.3867e+00, -4.6729e-01,  ..., -6.7734e+00,
          -8.0156e+00, -7.5000e+00],
         [-6.4492e+00,  2.7090e+00,  4.7500e+00,  ...,  4.3750e+00,
           5.2783e-01,  3.3965e+00],
         [-1.1141e+01, -4.4258e+00,  3.6426e+00,  ..., -5.2930e+00,
          -5.5977e+00, -2.5527e+00],
         ...,
         [-7.4805e+00, -1.0617e+01,  2.2520e+00,  ..., -5.1367e+00,
          -5.3906e+00, -3.2617e+00],
         [-3.9453e+00, -6.8945e+00,  5.7305e+00,  ..., -3.1074e+00,
          -7.5493e-03,  4.8682e-01],
         [-2.8789e+00, -3.5391e+00,  9.4062e+00,  ..., -1.0967e+00,
           4.6826e-01, -2.6978e-01]]])

In [12]:
r['logits']

tensor([[[-15.0938,  -5.7812,   2.3984,  ...,  -5.9688,  -8.0703,  -7.7617],
         [ -6.1367,   2.5801,   5.0078,  ...,   5.0117,   0.8262,   3.8789],
         [-10.2031,  -4.7773,   5.4688,  ...,  -3.6602,  -3.8125,  -0.4397],
         ...,
         [ -7.9609, -11.2344,   1.6582,  ...,  -5.3398,  -5.9180,  -3.7461],
         [ -3.5176,  -6.5391,   5.8984,  ...,  -2.7578,   0.0967,   0.9409],
         [ -2.7266,  -3.2832,   9.0859,  ...,  -0.5991,   0.4150,  -0.1166]]])

In [13]:
model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.36.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [18]:
model.model._use_sdpa

True

In [19]:
type(model.model.layers[0].self_attn)

transformers.models.llama.modeling_llama.LlamaSdpaAttention